<a href="https://colab.research.google.com/github/ThibeVanOrshaegen/.github/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/nothings/stb.git

Cloning into 'stb'...
remote: Enumerating objects: 8031, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 8031 (delta 99), reused 105 (delta 78), pack-reused 7868
Receiving objects: 100% (8031/8031), 5.59 MiB | 10.31 MiB/s, done.
Resolving deltas: 100% (5324/5324), done.


In [2]:
!ls
!mkdir Cuda_Image
!mv /content/stb/stb_image*.* Cuda_Image

sample_data  stb


In [3]:
!wget -O skyline.jpg https://images-prod-uitdatabank.imgix.net/329eb7c3-df7c-413d-9196-955fd13526af.jpeg
%cd Cuda_Image
!ls

--2024-02-20 12:33:47--  https://images-prod-uitdatabank.imgix.net/329eb7c3-df7c-413d-9196-955fd13526af.jpeg
Resolving images-prod-uitdatabank.imgix.net (images-prod-uitdatabank.imgix.net)... 151.101.2.208, 151.101.66.208, 151.101.130.208, ...
Connecting to images-prod-uitdatabank.imgix.net (images-prod-uitdatabank.imgix.net)|151.101.2.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2427278 (2.3M) [image/jpeg]
Saving to: ‘skyline.jpg’

skyline.jpg         100%[===================>]   2.31M  --.-KB/s    in 0.01s   

2024-02-20 12:33:48 (173 MB/s) - ‘skyline.jpg’ saved [2427278/2427278]

/content/Cuda_Image
stb_image.h  stb_image_resize2.h  stb_image_write.h


In [4]:
%%file image_read.c

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

int main(void)
{
    int width, height, channels;
    unsigned char *img = stbi_load("skyline.jpg", &width, &height, &channels, 0);
    if(img == NULL) {
          printf("Error in loading the image\n");
          exit(1);
     }
    printf("Loaded image with a width of %dpx, a height of %dpx and %d channels\n", width, height, channels);
    return 0;
}

Writing image_read.c


In [5]:
!gcc -o -Wall -o image_read image_read.c -lm

In [6]:
!./image_read

Error in loading the image


In [7]:
%%file image_read_write_jpg2png.c

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

int main(void)
{
    int width, height, channels;
    unsigned char *img = stbi_load("../skyline.jpg", &width, &height, &channels, 0);
    if(img == NULL) {
          printf("Error in loading the image\n");
          exit(1);
     }
    printf("Loaded image with a width of %dpx, a height of %dpx and %d channels\n", width, height, channels);
    stbi_write_png("skyline.png",width,height,channels,img,width*channels);
    stbi_image_free(img);
    return 0;
}

Writing image_read_write_jpg2png.c


In [8]:
!gcc -o -Wall -o image_read_write_jpg2png image_read_write_jpg2png.c -lm
!./image_read_write_jpg2png

Loaded image with a width of 3648px, a height of 2736px and 3 channels


In [9]:
%%file image_jpg_color2gray.c

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

int main(void)
{
    int width, height, channels;
    unsigned char *img = stbi_load("../skyline.jpg", &width, &height, &channels, 0);
    if(img == NULL) {
          printf("Error in loading the image\n");
          exit(1);
     }
    printf("Loaded image with a width of %dpx, a height of %dpx and %d channels\n", width, height, channels);

    // Convert the input image to gray
    size_t img_size = width * height * channels;
    int gray_channels = channels == 4 ? 2 : 1;
    size_t gray_img_size = width * height * gray_channels;

    unsigned char *gray_img = malloc(gray_img_size);
    if(gray_img == NULL) {
        printf("Unable to allocate memory for the gray image.\n");
        exit(1);
    }

    for(unsigned char *p = img, *pg = gray_img; p != img + img_size; p += channels, pg += gray_channels) {
        *pg = (uint8_t)((*p + *(p + 1) + *(p + 2))/3.0);
        if(channels == 4) {
            *(pg + 1) = *(p + 3);
        }
    }
    stbi_write_jpg("skyline_gray.jpg", width, height, gray_channels, gray_img, 100);
    stbi_image_free(img);
    return 0;
}

Writing image_jpg_color2gray.c


In [10]:
!gcc -o -Wall -o image_jpg_color2gray image_jpg_color2gray.c -lm
!./image_jpg_color2gray

Loaded image with a width of 3648px, a height of 2736px and 3 channels


In [11]:
%%timeit
!./image_jpg_color2gray

Loaded image with a width of 3648px, a height of 2736px and 3 channels
Loaded image with a width of 3648px, a height of 2736px and 3 channels
Loaded image with a width of 3648px, a height of 2736px and 3 channels
Loaded image with a width of 3648px, a height of 2736px and 3 channels
Loaded image with a width of 3648px, a height of 2736px and 3 channels
Loaded image with a width of 3648px, a height of 2736px and 3 channels
Loaded image with a width of 3648px, a height of 2736px and 3 channels
Loaded image with a width of 3648px, a height of 2736px and 3 channels
1.81 s ± 143 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
